<a href="https://colab.research.google.com/github/viniciusrpb/116319_estruturasdedados/blob/main/meteorological_data_analysis/meteorological_data_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [68]:
import pandas as pd
from sklearn import preprocessing
import numpy as np
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dense,LSTM,Lambda,RepeatVector,TimeDistributed
from keras.models import Model
import tensorflow as tf
from keras import backend as K

In [69]:
jf_raw_data = pd.read_csv("juiz_fora.csv",sep=';',decimal=',').dropna()
dourados_raw_data = pd.read_csv("dourados.csv",sep=';',decimal=',').dropna()

In [70]:
jf_raw_data

,Data,Hora (UTC),Temp. Ins. (C),Temp. Max. (C),Temp. Min. (C),Umi. Ins. (%),Umi. Max. (%),Umi. Min. (%),Pto Orvalho Ins. (C),Pto Orvalho Max. (C),Pto Orvalho Min. (C),Pressao Ins. (hPa),Pressao Max. (hPa),Pressao Min. (hPa),Vel. Vento (m/s),Dir. Vento (m/s),Raj. Vento (m/s),Radiacao (KJ/m²),Chuva (mm)
2,02/02/2022,200,19.5,19.8,19.5,97.0,97.0,97.0,19.1,19.4,19.1,911.3,911.5,911.3,2.3,167.0,3.6,0.7,0.0
3,02/02/2022,300,19.4,19.5,19.2,98.0,98.0,97.0,19.0,19.1,18.8,910.7,911.3,910.7,1.1,106.0,5.4,1.0,1.6
4,02/02/2022,400,19.4,19.5,19.3,98.0,98.0,98.0,19.0,19.1,18.9,910.1,910.7,910.0,3.1,95.0,4.5,0.8,0.4
5,02/02/2022,500,19.3,19.5,19.3,98.0,98.0,98.0,18.9,19.1,18.9,909.8,910.1,909.8,0.8,67.0,4.2,1.8,0.0
8,02/02/2022,800,19.0,19.3,19.0,98.0,98.0,98.0,18.6,18.9,18.6,910.0,910.0,909.6,2.1,183.0,3.9,0.6,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4481,07/08/2022,1700,23.0,23.3,22.2,59.0,62.0,59.0,14.7,14.9,14.2,909.7,910.4,909.7,3.2,78.0,6.3,1367.0,0.0
4482,07/08/2022,1800,23.2,24.3,22.9,60.0,60.0,56.0,14.9,15.2,14.4,908.9,909.7,908.9,3.3,29.0,7.2,1590.6,0.0
4483,07/08/2022,1900,23.2,23.4,23.0,58.0,60.0,57.0,14.4,14.9,14.2,909.0,909.1,908.9,2.2,55.0,7.2,734.9,0.0
4484,07/08/2022,2000,22.5,23.3,22.5,59.0,59.0,56.0,14.0,14.5,14.0,909.7,909.9,909.0,3.8,154.0,5.6,418.1,0.0


In [71]:
frames = [jf_raw_data,dourados_raw_data]

df_2 = pd.concat(frames)

df = df_2.replace(',','.')

df

,Data,Hora (UTC),Temp. Ins. (C),Temp. Max. (C),Temp. Min. (C),Umi. Ins. (%),Umi. Max. (%),Umi. Min. (%),Pto Orvalho Ins. (C),Pto Orvalho Max. (C),Pto Orvalho Min. (C),Pressao Ins. (hPa),Pressao Max. (hPa),Pressao Min. (hPa),Vel. Vento (m/s),Dir. Vento (m/s),Raj. Vento (m/s),Radiacao (KJ/m²),Chuva (mm)
2,02/02/2022,200,19.5,19.8,19.5,97.0,97.0,97.0,19.1,19.4,19.1,911.3,911.5,911.3,2.3,167.0,3.6,0.7,0.0
3,02/02/2022,300,19.4,19.5,19.2,98.0,98.0,97.0,19.0,19.1,18.8,910.7,911.3,910.7,1.1,106.0,5.4,1.0,1.6
4,02/02/2022,400,19.4,19.5,19.3,98.0,98.0,98.0,19.0,19.1,18.9,910.1,910.7,910.0,3.1,95.0,4.5,0.8,0.4
5,02/02/2022,500,19.3,19.5,19.3,98.0,98.0,98.0,18.9,19.1,18.9,909.8,910.1,909.8,0.8,67.0,4.2,1.8,0.0
8,02/02/2022,800,19.0,19.3,19.0,98.0,98.0,98.0,18.6,18.9,18.6,910.0,910.0,909.6,2.1,183.0,3.9,0.6,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4029,19/07/2022,2100,27.9,30.1,27.9,32.0,32.0,28.0,9.6,10.9,9.2,956.3,956.4,956.3,0.0,34.0,5.7,510.1,0.0
4030,19/07/2022,2200,25.0,27.9,25.0,41.0,41.0,32.0,10.8,10.8,9.5,956.5,956.6,956.3,0.0,34.0,0.0,23.1,0.0
4043,20/07/2022,1100,21.2,21.3,20.2,49.0,52.0,48.0,10.1,10.2,9.9,956.5,956.5,955.9,5.0,34.0,9.0,34.5,0.0
4044,20/07/2022,1200,23.5,23.5,21.2,44.0,50.0,43.0,10.6,10.6,10.1,956.9,956.9,956.5,5.3,34.0,9.1,543.2,0.0


In [72]:
label = df['Chuva (mm)']

flag_chuva = []
for label in label:
    if label > 0.0:
        flag_chuva.append(1)
    else:
        flag_chuva.append(0)

#df = df.drop(['Chuva (mm)'],axis=1)
y = np.array(flag_chuva)

Dummy Variables of Date and time

In [73]:
df['Hora (UTC)'] = pd.Categorical(df['Hora (UTC)'])
df_num = pd.get_dummies(df)

In [81]:
X = df_num.to_numpy()

sX = minmax_scale(X, axis = 0)

X_train, X_test, Y_train, Y_test = train_test_split(sX, y, train_size = 0.7, random_state = 42)

# encoder
latent_dim = 1
inter_dim = 32
timesteps, features = 228, 1

def sampling(args):
    z_mean, z_log_sigma = args
    batch_size = tf.shape(z_mean)[0]
    epsilon = K.random_normal(shape=(batch_size, latent_dim), mean=0., stddev=1.)
    return z_mean + z_log_sigma * epsilon

def vae_loss2(input_x, decoder1, z_log_sigma, z_mean):
    """ Calculate loss = reconstruction loss + KL loss for each data in minibatch """
    # E[log P(X|z)]
    recon = K.sum(K.binary_crossentropy(input_x, decoder1))
    # D_KL(Q(z|X) || P(z|X)); calculate in closed form as both dist. are Gaussian
    kl = 0.5 * K.sum(K.exp(z_log_sigma) + K.square(z_mean) - 1. - z_log_sigma)

# timesteps, features
input_x = Input(shape= (timesteps, features)) 

#intermediate dimension 
h = LSTM(inter_dim, activation='relu')(input_x)

#z_layer
z_mean = Dense(latent_dim)(h)
z_log_sigma = Dense(latent_dim)(h)
z = Lambda(sampling)([z_mean, z_log_sigma])

# Reconstruction decoder
decoder1 = RepeatVector(timesteps)(z)
decoder1 = LSTM(inter_dim, activation='relu', return_sequences=True)(decoder1)
decoder1 = TimeDistributed(Dense(features))(decoder1)

In [98]:
m = Model(input_x, decoder1)
#m.add_loss(vae_loss2(input_x, decoder1, z_log_sigma, z_mean)) #<===========
m.compile(loss = 'mse', optimizer='adam')

In [ ]:
m.fit(X_train, X_train, epochs = 100, batch_size = 100, shuffle = True, validation_data = (X_test, X_test))

Epoch 1/100
31/31 [==============================] - 11s 260ms/step - loss: 0.0307 - val_loss: 0.0267
Epoch 2/100
31/31 [==============================] - 8s 244ms/step - loss: 0.0206 - val_loss: 0.0153
Epoch 3/100
31/31 [==============================] - 8s 244ms/step - loss: 0.0149 - val_loss: 0.0146
Epoch 4/100
31/31 [==============================] - 8s 245ms/step - loss: 0.0144 - val_loss: 0.0141
Epoch 5/100
31/31 [==============================] - 8s 245ms/step - loss: 0.0142 - val_loss: 0.0140
Epoch 6/100
31/31 [==============================] - 8s 245ms/step - loss: 0.0141 - val_loss: 0.0139
Epoch 7/100
31/31 [==============================] - 8s 243ms/step - loss: 0.0140 - val_loss: 0.0138
Epoch 8/100
31/31 [==============================] - 8s 245ms/step - loss: 0.0144 - val_loss: 0.0140
Epoch 9/100
31/31 [==============================] - 8s 247ms/step - loss: 0.0140 - val_loss: 0.0138
Epoch 10/100
31/31 [==============================] - 8s 268ms/step - loss: 0.0139 - val_l